In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def make_line_data(size = 24, padding = 2, sparse = 2, rough = 0.5, max_turn = 1000):
    SIZE = size
    SIZE_1 = SIZE - 1
    data = np.zeros((SIZE,SIZE),dtype = int)
    startx = random.randint(0,SIZE_1)
    starty = random.randint(0,SIZE_1)
    data[startx,starty] = 1

    
    def is_recent(x_,y_,should_bigger_than):
        if x_ < 0 or x_ >= SIZE or y_ < 0 or y_ >= SIZE:
            return True
        if data[x_,y_] == 0:
            return True
        return data[x_,y_] > should_bigger_than
    k = 2

    def make_random_direction():
        random_direction_x = 0
        random_direction_y = random.randint(0,1) * 2 - 1
        if random.randint(0,1):
            random_direction_x, random_direction_y = random_direction_y, random_direction_x
        return random_direction_x, random_direction_y

    random_direction_x, random_direction_y = make_random_direction()
    failtime = 0
    turn_flag = True
    for i in range(2,SIZE * SIZE):
        if failtime > 30:
            break
        if(random.random() < rough):
            random_direction_x, random_direction_y = make_random_direction()
            turn_flag = True
        next_x = startx + random_direction_x
        next_y = starty + random_direction_y
        
        if(0 <= next_x < SIZE and 0 <= next_y < SIZE):
            should_bigger_than = data[startx,starty] - (2*sparse + 1)
            if data[next_x,next_y] == 0 and all([is_recent(next_x + m,next_y + n,should_bigger_than) for m in range(-sparse, sparse + 1) for n in range(-sparse, sparse + 1)]):
                data[next_x,next_y] = k
                k += 1
                startx = next_x
                starty = next_y
                failtime = 0
                if turn_flag:
                    turn_flag = False
                    max_turn -= 1
                    if(max_turn == 0):
                        break
            else :
                random_direction_x, random_direction_y = make_random_direction()
                failtime += 1
                turn_flag = True
        else:
            random_direction_x, random_direction_y = make_random_direction()
            failtime += 1
            turn_flag = True
    data = data.astype(bool).astype(int)
    data = np.pad(data,((padding,padding),(padding,padding)),'constant',constant_values = 0)
    return data


def make_hole_data(size = 28, sparse = 1, holesize = 2, padding = 2):
    SIZE = size
    SIZE_1 = SIZE - 1
    padding = padding + 1 + holesize
    data = make_line_data(sparse = sparse + 2, size=size - 2*padding, padding = padding)
    for k in range(holesize):
        data_ = data.copy()
        for i in range(SIZE):
            for j in range(SIZE):
                if data[i,j] == 1:
                    for ii in range(i-1,i+2):
                        for jj in range(j-1,j+2):
                            if 0 <= ii < SIZE and 0 <= jj < SIZE and data[ii,jj] == 0:
                                data_[ii,jj] = 1
        data = data_
    for i in range(SIZE):
        for j in range(SIZE):
            if data[i,j] == 1:
                for ii in range(i-1,i+2):
                    for jj in range(j-1,j+2):
                        if 0 <= ii < SIZE and 0 <= jj < SIZE and data[ii,jj] == 0:
                            data[ii,jj] = 2
    m = [0,0,1]
    for i in range(SIZE):
        for j in range(SIZE):
            data[i,j] = m[data[i,j]]
    return data

def make_datas(n):
    data = []
    for i in range(n//50):
        for sparse in range(1,6):
            for roughi in range(2,12,2):
                data.append((make_line_data(sparse=sparse, rough = roughi*0.05, max_turn = roughi * 10), 1))

    for i in range(n//50):
        for sparse in range(1,6):
            for holesize in range(0,5):
                data.append((make_hole_data(sparse=sparse, holesize=holesize), 0))
    
    random.shuffle(data)
    ret_data,ret_label = zip(*data)
    ret_data = np.array(ret_data)
    ret_label = np.array(ret_label)
    return ret_data,ret_label
train_images, train_labels = make_datas(60000)
test_images, test_labels = make_datas(10000)

np.savez_compressed('cycle_dataset.npz', train_images = train_images, train_labels = train_labels, test_images = test_images, test_labels = test_labels)

In [ ]:
'''
# how to load and plot

cycle_dataset = np.load('cycle_dataset.npz')
train_images = cycle_dataset['train_images']
train_labels = cycle_dataset['train_labels']
test_images = cycle_dataset['test_images']
test_labels = cycle_dataset['test_labels']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()
'''